In [2]:
!pip install -U langchain-huggingface

In [3]:
import os
from langchain_community.document_loaders import TextLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain_huggingface import HuggingFaceEmbeddings, HuggingFacePipeline
from langchain.vectorstores import Chroma
from langchain.chains import RetrievalQA
from transformers import pipeline

# 🔹 Suppress Hugging Face parallelism warning
os.environ["TOKENIZERS_PARALLELISM"] = "false"

# 🔹 Load a Hugging Face model for text generation
qa_model = pipeline(
    "text-generation",
    model="facebook/opt-1.3b",
    max_new_tokens=100,  # Fix max_length error
    pad_token_id=50256  # Prevents padding token issues
)

# 🔹 Sample document text (simulating a policy file)
sample_text = """The new policy states that all government agencies must undergo a security review before deploying AI models.
This ensures compliance with national security standards and protects sensitive information.
Agencies must also document AI decision-making processes for transparency and accountability."""

# 🔹 Save this text as a document
with open("sample_policy.txt", "w") as file:
    file.write(sample_text)

# 🔹 Load the text document
loader = TextLoader("sample_policy.txt")
documents = loader.load()

# 🔹 Split text into smaller chunks
text_splitter = CharacterTextSplitter(chunk_size=100, chunk_overlap=20)
docs = text_splitter.split_documents(documents)

# 🔹 Create vector embeddings from the text
embedding_function = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
vectorstore = Chroma.from_documents(docs, embedding_function)

# 🔹 Set up the document retriever
retriever = vectorstore.as_retriever()

# 🔹 Create a RAG-powered Q&A system
qa_chain = RetrievalQA.from_chain_type(llm=HuggingFacePipeline(pipeline=qa_model), retriever=retriever)

# 🔹 Ask a question based on the document
query = "What is the main policy discussed in this document?"
response = qa_chain.invoke(query)  # Fixed deprecated `.run()` method

# 🔹 Print the AI-generated answer
print("AI Answer:", response)

Device set to use mps:0
Number of requested results 4 is greater than number of elements in index 2, updating n_results = 2


AI Answer: {'query': 'What is the main policy discussed in this document?', 'result': "Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.\n\nThe new policy states that all government agencies must undergo a security review before deploying AI models.\nThis ensures compliance with national security standards and protects sensitive information.\nAgencies must also document AI decision-making processes for transparency and accountability.\n\nThe new policy states that all government agencies must undergo a security review before deploying AI models.\nThis ensures compliance with national security standards and protects sensitive information.\nAgencies must also document AI decision-making processes for transparency and accountability.\n\nQuestion: What is the main policy discussed in this document?\nHelpful Answer:\n\nThe new policy states that all government agencies must under

In [5]:
response = qa_chain.invoke(query)
print("AI Answer:", response['result'])  # Extract only the final answer

Number of requested results 4 is greater than number of elements in index 2, updating n_results = 2


AI Answer: Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.

The new policy states that all government agencies must undergo a security review before deploying AI models.
This ensures compliance with national security standards and protects sensitive information.
Agencies must also document AI decision-making processes for transparency and accountability.

The new policy states that all government agencies must undergo a security review before deploying AI models.
This ensures compliance with national security standards and protects sensitive information.
Agencies must also document AI decision-making processes for transparency and accountability.

Question: What are the AI security requirements in this policy?
Helpful Answer:

The new policy states that all government agencies must undergo a security review before deploying AI models.
This ensures compliance with national 